In [6]:
from tqdm import tqdm
from analysis.nlp_transformations import separate_previous_conversation, remove_footers
from typing import List

from dao.email import DAOEmailGenerated, DAORealEmail
from dao.attribute import DAOAttribute

from analysis.nlp_transformations import split_into_sentences, replace_links_with_text

/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[nltk_data] Downloading package stopwords to /home/pawel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pawel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package pl196x to /home/pawel/nltk_data...
[nltk_data]   Package pl196x is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pawel/nltk_data...
[nltk_data]   Packa

In [7]:
text = "Oto kilkanaście zdań w języku polskim, które zawierają różnorodne formy interpunkcyjne i struktury zdaniowe. Jak się dzisiaj czujesz? Dlaczego niebo jest niebieskie? Czy wiesz, że Ks. Jan Twardowski był znanym polskim poetą? Uwaga! Proszę nie biec po korytarzu. Gdzie jest najbliższa apteka? Ile to kosztuje? To niesamowite, że możemy rozmawiać online! Co sądzisz o najnowszych odkryciach w kosmosie? Pamiętaj, że drzwi otwierają się o godz. 8:00 rano! Czy możesz mi podać sól? Kto zostanie nowym prezydentem Stanów Zjednoczonych? Wow! To było naprawdę ekscytujące. Jak możemy efektywniej oszczędzać energię? Czy znasz jakieś dobre przepisy na ciasto? Dr M. Nowak będzie dziś wygłaszać wykład na uniwersytecie. Czy myślisz, że sztuczna inteligencja może przewyższyć ludzką kreatywność?"
one_sentence = "I eat breakfast at 8:00 a.m."
split_into_sentences(text, 'pl')

['Oto kilkanaście zdań w języku polskim, które zawierają różnorodne formy interpunkcyjne i struktury zdaniowe.',
 'Jak się dzisiaj czujesz?',
 'Dlaczego niebo jest niebieskie?',
 'Czy wiesz, że Ks. Jan Twardowski był znanym polskim poetą?',
 'Uwaga!',
 'Proszę nie biec po korytarzu.',
 'Gdzie jest najbliższa apteka?',
 'Ile to kosztuje?',
 'To niesamowite, że możemy rozmawiać online!',
 'Co sądzisz o najnowszych odkryciach w kosmosie?',
 'Pamiętaj, że drzwi otwierają się o godz. 8:00 rano!',
 'Czy możesz mi podać sól?',
 'Kto zostanie nowym prezydentem Stanów Zjednoczonych?',
 'Wow!',
 'To było naprawdę ekscytujące.',
 'Jak możemy efektywniej oszczędzać energię?',
 'Czy znasz jakieś dobre przepisy na ciasto?',
 'Dr M. Nowak będzie dziś wygłaszać wykład na uniwersytecie.',
 'Czy myślisz, że sztuczna inteligencja może przewyższyć ludzką kreatywność?']

In [9]:
dao_attribute = DAOAttribute()
dao_real_email = DAORealEmail()
dao_generated_email = DAOEmailGenerated()

dao_generated_emails: DAOEmailGenerated = DAOEmailGenerated()
dao_real_emails: DAORealEmail = DAORealEmail()
dao_attribute: DAOAttribute = DAOAttribute()


all_attributes: List[AttributeInDB] = dao_attribute.find_many_by_query({'average_word_char_length': None})

for attribute in tqdm(all_attributes, total=len(all_attributes), desc=f'Calculating sentences statistics', unit='emails', miniters=1):
    if attribute.referenced_db_name == 'email_generated_dataset':
        og_email = dao_generated_emails.find_one_by_query({'_id': attribute.referenced_doc_id})
        language = og_email.language
    else:
        og_email = dao_real_emails.find_one(attribute.referenced_db_name, {'_id': attribute.referenced_doc_id})
        language = og_email.detected_language
    _, og_text = separate_previous_conversation(og_email.text_plain)
    og_text = remove_footers(og_text)
    og_text = replace_links_with_text(og_text, "")
    
    words = [token for token in og_text.split() if token not in string.punctuation]
    average_word_char_length = sum(len(word) for word in words) / len(words)
    
    punctuation = len([char for char in og_text if char in string.punctuation])
    
    split_sentences: List[str] = split_into_sentences(og_text, language)
    number_of_sentences = len(split_sentences)
    char_length_distribution, word_length_distribution = calc_distribution_sentence_length(split_sentences) 
    
    dao_attribute.update_one({'_id': attribute.id}, {'$set':{
        'punctuation': punctuation,
        'number_of_sentences': number_of_sentences,
        'average_word_char_length': average_word_char_length,
        
        'standard_deviation_sentence_word_length': word_length_distribution[0],
        'variance_sentence_word_length':word_length_distribution[1],
        'average_sentence_word_length': word_length_distribution[2],
        
        'standard_deviation_sentence_char_length': char_length_distribution[0],
        'variance_sentence_char_length': char_length_distribution[1],
        'average_sentence_char_length': char_length_distribution[2],
    
    }})


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66089a91c7a3ff1b3a95b45b, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>